In [1]:
import xgboost as xgb
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
import sklearn.model_selection as ms
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score as im
from sklearn.svm import SVC
from sklearn.metrics import mean_absolute_error
import shap as shap
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
import boruta as b

In [2]:
audio = pd.read_csv(r"audio.csv")
audio = audio.drop(['Unnamed: 0', 'Unnamed: 0_x'], axis = 1)

In [3]:
y = audio['sex'].replace('F',0)
y = y.replace('M',1)
X = audio.iloc[:,3:119]


In [4]:

X_train, X_test, y_train, y_test = ms.train_test_split(X,y, test_size = 0.25, random_state = 12)


In [5]:
classifier_svc = SVC()
classifier_svc.fit(X_train, y_train)
y_pred_svc = classifier_svc.predict(X_test)
score = im(y_pred_svc,y_test)
print("score_svc  : ", im(y_pred_svc,y_test))

classifier = RandomForestClassifier(n_estimators = 120)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
#print("y_train[:] ",y_train[:] )
print("score_random_forest  : ", im(y_pred,y_test))


clf_xgb = xgb.XGBClassifier()
clf_xgb.fit(X_train, y_train)
y_pred_xgb = clf_xgb.predict(X_test)

#Calcul de la performance de nos predictions
print("score_xgb  : ",im(y_pred_xgb,y_test))

score_svc  :  0.5890227576974565
score_random_forest  :  0.5796519410977242
score_xgb  :  0.5622489959839357


On constate que pour les paramètres par défaut le randomforest est meilleur que svc et xgboost. Il a un score de 0.60.

On procède à la création des fonctions qui vont nous permettre de voir l'impact des différents paramètres pour chacun de ces algo.

In [6]:
def svc(kernel,  X_train, y_train, X_test, y_test):
    classifier_svc = SVC(kernel=kernel)
    classifier_svc.fit(X_train, y_train)
    y_pred_svc = classifier_svc.predict(X_test)
    score = im(y_pred_svc,y_test)
    return score

In [7]:
def xgbClassifier(n_estimators, X_train, y_train, X_test, y_test):
    clf_xgb = xgb.XGBClassifier(n_estimators=n_estimators)
    clf_xgb.fit(X_train, y_train)
    y_pred_xgb = clf_xgb.predict(X_test)
    score = im(y_pred_xgb,y_test)
    return score
    
    

scores_gini = []
scores_entropy = []
for i in range(100,150):
    clf_g = RandomForestClassifier(n_estimators=i, criterion = 'gini')
    clf_g = clf_g.fit(X_train, y_train)
    ypred_g = clf_g.predict(X_test)
    score = im(ypred_g,y_test)
    scores_gini.append(score)
for i in range(100,150):
    clf = RandomForestClassifier(n_estimators=i, criterion = 'entropy')
    clf = clf.fit(X_train, y_train)
    ypred = clf.predict(X_test)
    score = im(ypred,y_test)
    scores_entropy.append(random_forest(i,'entropy', X_train, y_train, X_test, y_test))
print(scores_gini)
print(scores_entropy)

In [8]:
rf =  RandomForestClassifier(n_estimators= 120)

In [ ]:
feat_selector = b.BorutaPy(rf, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X.values, y.values)

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	116
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	116
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	116
Rejected: 	0


Grace à l'algorithme de boruta, on arrive à réduire le nombre de colonnes en ne gardant que celle qui sont significatives dans notre prédiction.

In [ ]:
liste = feat_selector.support_
good_col = []
for i in range(len(liste)):
    if liste[i]==True:
        good_col.append(i)
good_col

In [ ]:
new_X = audio[audio.columns[good_col]].values

In [ ]:
rf.fit(X_train,y_train)
shap_explainer = shap.TreeExplainer(rf)
shap_values = shap_explainer.shap_values(new_X)
shap.summary_plot(shap_values[0], X, plot_type = 'dot' )

Partie 2

Analyse prédictive de l’âge

In [ ]:
X_age = audio.iloc[:,3:119]
y = audio['birth_date']
y

In [ ]:
X_train, X_test, y_train, y_test = ms.train_test_split(X_age,y, test_size = 0.25, random_state = 12)
X_train

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
scaler = StandardScaler()
scaler.fit_transform(X_train, y_train)

In [ ]:
classifier_mlp = MLPRegressor(max_iter=10000)
classifier_mlp.fit(X_train, y_train)
y_pred_mlp = classifier_mlp.predict(X_test)
score = mean_absolute_error(y_pred_mlp,y_test)
print("score_mlp  : ", mean_absolute_error(y_pred_mlp,y_test))

classifier = RandomForestRegressor()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)
#print("y_train[:] ",y_train[:] )
print("score_random_forest  : ", mean_absolute_error(y_pred,y_test))


clf_xgb = xgb.XGBRegressor()
clf_xgb.fit(X_train, y_train)
y_pred_xgb = clf_xgb.predict(X_test)

#Calcul de la performance de nos predictions
print("score_xgb  : ",mean_absolute_error(y_pred_xgb, y_test))

On observe une erreur minimum pour l'algorithme de randomforest. L'algorithme des reseaux de neurones est éliminé.
Maintenant, on va essayer de minimiser l'erreur en jouant sur les paramètres de chaque algorithme.

In [ ]:
score={}
for i in ['lbfgs', 'sgd', 'adam']:
    classifier_mlp = MLPRegressor(max_iter=10000, solver=i)
    classifier_mlp.fit(X_train, y_train)
    y_pred_mlp = classifier_mlp.predict(X_test)
    score[i] = mean_absolute_error(y_pred_mlp,y_test)
    print("score_mlp  : ", mean_absolute_error(y_pred_mlp,y_test))


In [ ]:
score_rf= []
for i in range(100,150):
    classifier = RandomForestRegressor(n_estimators=i)
    classifier.fit(X_train, y_train)
    y_pred = classifier.predict(X_test)
    score_rf.append(mean_absolute_error(y_pred,y_test))
score_rf
    

In [ ]:
score_xgb= []
for i in range(100,150):
    clf_xgb = xgb.XGBRegressor(n_estimators=i)
    clf_xgb.fit(X_train, y_train)
    y_pred_xgb = clf_xgb.predict(X_test)
    score_xgb.append(mean_absolute_error(y_pred_xgb,y_test))
    
score_xgb

In [ ]:
best_model = 

In [ ]:
feat_selector = b.BorutaPy(best_model, n_estimators='auto', verbose=2, random_state=1)
feat_selector.fit(X, y)

In [ ]:
liste = feat_selector.support_
good_col = []
for i in range(len(liste)):
    if liste[i]==True:
        good_col.append(i)
good_col

In [ ]:
new_X = audio[audio.columns[good_col]].values

In [ ]:
shap_explainer = shap.TreeExplainer(best_model)
shap_values = shap_explainer.shap_values(new_X)
shap.summary_plot(shap_values[0], X, plot_type = 'dot' )

Analyse prédictive de la qualité audio 

L’objectif ici est l’inverse des analyses précédentes : vous devez prédire la capacité auditive
d’une fréquence audio à partir des données anthropologiques des individus.

In [ ]:

df= audio.loc[:,['population_id', 'birth_date','education', 'sex', 'environment_biotope', 'lang_group', 'laterality']]
df['lang_group']=df['lang_group'].fillna('other')
df['laterality']=df['laterality'].fillna('other')
df

In [ ]:
le = LabelEncoder()
df['education'] = le.fit_transform(df['education'])
df['sex'] = le.fit_transform(df['sex'])
df['environment_biotope'] = le.fit_transform(df['environment_biotope'])
df['lang_group'] = le.fit_transform(df['lang_group'])
df['laterality'] = le.fit_transform(df['laterality'])

In [ ]:
grave = audio.iloc[:,14]
medium = audio.iloc[:,30]
aigue = audio.iloc[:,80]

D'abord on va entrainer le modèle sur la colonne grave

In [ ]:
X_train_g, X_test_g, y_train_g, y_test_g = ms.train_test_split(df,grave, test_size = 0.25, random_state = 12)
X_train_g

In [ ]:
classifier = RandomForestRegressor()
classifier.fit(X_train_g, y_train_g)
y_pred = classifier.predict(X_test_g)
score_rf=mean_absolute_error(y_pred,y_test_g)

In [ ]:
score_rf

Avec randomforest, la moyenne des erreurs est de d'environ 7. ce qui est un score très bien.

In [ ]:
shap_explainer = shap.TreeExplainer(classifier)
shap_values = shap_explainer.shap_values(df)
shap.summary_plot(shap_values, df, plot_type = 'dot' )

Donc d'après le graphe des shapvalues, les colonnes qui impactent fortement la colonne des sons graves sont : birth_date, population_id,environment_biotope, lang_group et plus faiblement la colonne education. Les distributions des données pour la colonne sex et laterality sont très proches de l'axe verticale, ce qui signifie que l'impact de ces colonnes sur l'output très faible.

In [ ]:
X_train_m, X_test_m, y_train_m, y_test_m = ms.train_test_split(df,medium, test_size = 0.25, random_state = 12)
X_train_m

In [ ]:
classifier.fit(X_train_m, y_train_m)
y_pred_m = classifier.predict(X_test_m)
score_rf_m=mean_absolute_error(y_pred_m,y_test_m)

In [ ]:
score_rf_m

Pour la colonne des medium, la prédiction du random forest est meilleur.

In [ ]:
shap_explainer = shap.TreeExplainer(classifier)
shap_values = shap_explainer.shap_values(df)
shap.summary_plot(shap_values, df, plot_type = 'dot' )

Pour la colonne de medium, les colonnes très influentes classées de la plus influente à la moins influente sont lang_roup, birth_date, population_id, education et sex. L'environment_biotope a un imapct plus faible. la colonne laterality est négligeable dans la prédiction de cette colonne.

In [ ]:
X_train_a, X_test_a, y_train_a, y_test_a = ms.train_test_split(df,aigue, test_size = 0.25, random_state = 12)


In [ ]:
clf = RandomForestRegressor()
clf.fit(X_train_a, y_train_a)
y_pred_a = clf.predict(X_test_a)
score_rf_a=mean_absolute_error(y_pred_a,y_test_a)

In [ ]:
score_rf_a

Le score pour la colonne aigue est excellent car l'erreur moyenne est d'environ 6,60. 

In [ ]:
shap_explainer = shap.TreeExplainer(clf)
shap_values = shap_explainer.shap_values(df)
shap.summary_plot(shap_values, df, plot_type = 'dot' )

Pour la colonne aigue, les colonnes très influentes classées de la plus influente à la moins influente sont birth_date, sex, population_id, environment_biotope et education. La lang_roup a un imapct plus faible. la colonne laterality est négligeable dans la prédiction de cette colonne.